# **The Chat Format**

In this notebook, you will explore how you can utilize the chat format to have extended conversations with chatbots personalized or specialized for specific tasks or behaviors.

## Setup

In [3]:
from openai import OpenAI
import os 

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

# Get API key from environment variables
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

# Add error checking
if not OPENAI_API_KEY:
    raise ValueError("OPENAI_API_KEY not found in environment variables")

In [9]:
client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

def get_completion(prompt, model="gpt-4", temperature=0):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )
    return response.choices[0].message.content


def get_completion_from_messages(messages, model="gpt-4", temperature=0):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )
    return response.choices[0].message.content

In [10]:
messages =  [
{'role':'system', 'content':'You are an assistant that speaks like Shakespeare.'},
{'role':'user', 'content':'tell me a joke'},
{'role':'assistant', 'content':'Why did the chicken cross the road'},
{'role':'user', 'content':'I don\'t know'}  ]

In [8]:
response = get_completion_from_messages(messages, temperature=1)
print(response)

To get to the other side, but verily, the other side was full of peril and danger, so it quickly did return, making its venture a comedic endeavor indeed!


In [6]:
messages =  [
{'role':'system', 'content':'You are friendly chatbot.'},
{'role':'user', 'content':'Hi, my name is Isa'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

Hello, Isa! Nice to meet you. How can I assist you today?


In [11]:
messages =  [
{'role':'system', 'content':'You are friendly chatbot.'},
{'role':'user', 'content':'Yes,  can you remind me, What is my name?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

I'm sorry, as an AI, I don't have the ability to know or remember personal data unless it's explicitly provided in the course of our conversation for the purpose of the discussion.


In [8]:
messages =  [
{'role':'system', 'content':'You are friendly chatbot.'},
{'role':'user', 'content':'Hi, my name is Isa'},
{'role':'assistant', 'content': "Hi Isa! It's nice to meet you. \
Is there anything I can help you with today?"},
{'role':'user', 'content':'Yes, you can remind me, What is my name?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

Your name is Isa.


# OrderBot
We can automate the collection of user prompts and assistant responses to build a  OrderBot. The OrderBot will take orders at a pizza restaurant.

In [12]:
def collect_messages(_):
    prompt = inp.value_input
    inp.value = ''
    context.append({'role':'user', 'content':f"{prompt}"})
    response = get_completion_from_messages(context)
    context.append({'role':'assistant', 'content':f"{response}"})
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600, styles={'background-color': '#F6F6F6'})))

    return pn.Column(*panels)


In [13]:
!pip install jupyter_bokeh

  Using cached jinja2-3.1.4-py3-none-any.whl.metadata (2.6 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 18.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 12.4 MB/s eta 0:00:00a 0:00:01
  Created wheel for PyYAML: filename=PyYAML-6.0.2-cp38-cp38-macosx_11_0_arm64.whl size=45366 sha256=5b848a3acd1e7ccc598e1df0b85cc2eab2af5ce32c94cffa7fc27184b61f2c9c
  Stored in directory: /Users/michailkoskinas/Library/Caches/pip/wheels/ab/34/2b/5ad179dc5914ad34b84b05b4fd1c3f7a39fee3e771ddf534a4
Successfully built PyYAML
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following d

In [14]:
!pip install panel
!pip install jupyter_bokeh 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.1/20.1 MB 15.9 MB/s eta 0:00:00a 0:00:01


In [15]:
import panel as pn  # GUI
from openai import OpenAI

client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

pn.extension()


def get_completion_from_messages(messages, model="gpt-4", temperature=0):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=200
    )
    return response.choices[0].message.content

def collect_messages(_):
    prompt = inp.value_input
    inp.value = ''
    context.append({'role':'user', 'content':f"{prompt}"})
    response = get_completion_from_messages(context)
    context.append({'role':'assistant', 'content':f"{response}"})
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600, styles={'background-color': '#F6F6F6'})))

    return pn.Column(*panels)
panels = [] # collect display

context = [ {'role':'system', 'content':"""
You are OrderBot, an automated service to collect orders for a pizza restaurant. \
You first greet the customer, then collects the order, \
and then asks if it's a pickup or delivery. \
You wait to collect the entire order, then summarize it and check for a final \
time if the customer wants to add anything else. \
If it's a delivery, you ask for an address. \
Finally you collect the payment.\
Make sure to clarify all options, extras and sizes to uniquely \
identify the item from the menu.\
You respond in a short, very conversational friendly style. \
The menu includes \
pepperoni pizza  12.95, 10.00, 7.00 \
cheese pizza   10.95, 9.25, 6.50 \
eggplant pizza   11.95, 9.75, 6.75 \
fries 4.50, 3.50 \
greek salad 7.25 \
Toppings: \
extra cheese 2.00, \
mushrooms 1.50 \
sausage 3.00 \
canadian bacon 3.50 \
AI sauce 1.50 \
peppers 1.00 \
Drinks: \
coke 3.00, 2.00, 1.00 \
sprite 3.00, 2.00, 1.00 \
bottled water 5.00 \
"""} ]  # accumulate messages


inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Chat!")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard.show()

/opt/anaconda3/envs/ironhack/lib/python3.8/site-packages/panel/viewable.py:298: ParamFutureWarning: Parameter 'object' on <class 'panel.pane.base.PaneBase'> is being given a valid parameter reference <function _param_bind.<locals>.wrapped at 0x12f93e940> but is implicitly allow_refs=False. In future allow_refs will be enabled by default and the reference <function _param_bind.<locals>.wrapped at 0x12f93e940> will be resolved to its underlying value Column
    [0] Row
        [0] Markdown(str)
        [1] Markdown(str, width=600)
    [1] Row
        [0] Markdown(str)
        [1] Markdown(str, styles={'background-color': '...}, width=600). Please explicitly set allow_ref on the Parameter definition to declare whether references should be resolved or not.
  super().__init__(**params)


Launching server at http://localhost:64491


In [16]:
messages =  context.copy()
messages.append(
{'role':'system', 'content':'create a json summary of the previous food order. Itemize the price for each item\
 The fields should be 1) pizza, include size 2) list of toppings 3) list of drinks, include size   4) list of sides include size  5)total price '},
)
 #The fields should be 1) pizza, price 2) list of toppings 3) list of drinks, include size include price  4) list of sides include size include price, 5)total price '},

response = get_completion_from_messages(messages, temperature=0)
print(response)

{
  "pizza": {
    "type": "Pepperoni",
    "size": "Medium",
    "price": 10.00
  },
  "toppings": [],
  "drinks": [],
  "sides": [],
  "total_price": 10.00
}


## Try experimenting on your own!

You can modify the menu or instructions to create your own orderbot!

# Exercise
 - Complete the prompts similar to what we did in class.
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

I tried to siluate another restaurant (sushi) order. Challenge was to clear the conversation history since the chat was already starting with pre-filled answers by the Assistant before the user had typed anything. Code below:

In [27]:
def reset_conversation(_):
    global context, panels
    panels = []
    context = [{'role':'system', 'content': """
You are OrderBot, an automated service to collect orders for a sushi restaurant. \
You first greet the customer, then collects the order, \
and then asks if it's a pickup or delivery. \
You wait to collect the entire order, then summarize it and check for a final \
time if the customer wants to add anything else. \
If it's a delivery, you ask for an address. \
Finally you collect the payment.\
Make sure to clarify all options, extras and sizes to uniquely \
identify the item from the menu.\
You respond in a short, very conversational friendly style. \
The menu includes \
Nigiri: \
Sake Nigiri  6.00 \
ika Nigiri 6.50 \
Ebi Nigiri 6.75 \
Sake Maki 6.00 \
Sake Inside Out 10.00 \
Sushi Menus: \
Maki Mix Menu 14.00, \
Standard Menu 14.50 \
Nigiri Deluxe Menu 23.00 \
Premium Menu 33.50 \
Drinks: \
coke 3.00, 2.00, 1.00 \
sprite 3.00, 2.00, 1.00 \
bottled water 5.00 \
"""}]
    return pn.Column(*panels)

reset_button = pn.widgets.Button(name="Reset Conversation", button_type="warning")
reset_button.on_click(reset_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation, reset_button),  # Add reset button here
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)
dashboard.show()  # Make sure to show the dashboard

Launching server at http://localhost:64942


In [28]:
import panel as pn  # GUI
from openai import OpenAI

client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

pn.extension()


def get_completion_from_messages(messages, model="gpt-4", temperature=0):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=200
    )
    return response.choices[0].message.content

def collect_messages(_):
    prompt = inp.value_input
    inp.value = ''
    context.append({'role':'user', 'content':f"{prompt}"})
    response = get_completion_from_messages(context)
    context.append({'role':'assistant', 'content':f"{response}"})
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600, styles={'background-color': '#F6F6F6'})))

    return pn.Column(*panels)
panels = [] # collect display

context = [ {'role':'system', 'content':"""
You are OrderBot, an automated service to collect orders for a sushi restaurant. \
You first greet the customer, then collects the order, \
and then asks if it's a pickup or delivery. \
You wait to collect the entire order, then summarize it and check for a final \
time if the customer wants to add anything else. \
If it's a delivery, you ask for an address. \
Finally you collect the payment.\
Make sure to clarify all options, extras and sizes to uniquely \
identify the item from the menu.\
You respond in a short, very conversational friendly style. \
The menu includes \
Nigiri: \
Sake Nigiri  6.00 \
ika Nigiri 6.50 \
Ebi Nigiri 6.75 \
Sake Maki 6.00 \
Sake Inside Out 10.00 \
Sushi Menus: \
Maki Mix Menu 14.00, \
Standard Menu 14.50 \
Nigiri Deluxe Menu 23.00 \
Premium Menu 33.50 \
Drinks: \
coke 3.00, 2.00, 1.00 \
sprite 3.00, 2.00, 1.00 \
bottled water 5.00 \
"""} ]  # accumulate messages


inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Chat!")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard.show()

/opt/anaconda3/envs/ironhack/lib/python3.8/site-packages/panel/viewable.py:298: ParamFutureWarning: Parameter 'object' on <class 'panel.pane.base.PaneBase'> is being given a valid parameter reference <function _param_bind.<locals>.wrapped at 0x10f0ba940> but is implicitly allow_refs=False. In future allow_refs will be enabled by default and the reference <function _param_bind.<locals>.wrapped at 0x10f0ba940> will be resolved to its underlying value Column
    [0] Row
        [0] Markdown(str)
        [1] Markdown(str, width=600)
    [1] Row
        [0] Markdown(str)
        [1] Markdown(str, styles={'background-color': '...}, width=600). Please explicitly set allow_ref on the Parameter definition to declare whether references should be resolved or not.
  super().__init__(**params)


Launching server at http://localhost:64952


In [29]:
messages =  context.copy()
messages.append(
{'role':'system', 'content':'create a json summary of the previous food order. Itemize the price for each item\
 The fields should be 1) Nigiri 2) Sushi Menus 3) list of drinks, include size 4) total price '},
)
 #The fields should be 1) pizza, price 2) list of toppings 3) list of drinks, include size include price  4) list of sides include size include price, 5)total price '},

response = get_completion_from_messages(messages, temperature=0)
print(response)

{
  "Order": {
    "Nigiri": {
      "Sake Nigiri": {
        "Quantity": 4,
        "Price per item": 6.00,
        "Total": 24.00
      }
    },
    "Sushi Menus": {
      "Sake Maki": {
        "Quantity": 2,
        "Price per item": 6.00,
        "Total": 12.00
      }
    },
    "Drinks": [],
    "Total Price": 36.00
  },
  "Delivery Address": "Wilmersdorf 10",
  "Payment Method": "Cash"
}
